In [ ]:
!pip install vectorbt

In [2]:
import vectorbt as vbt
import yfinance as yf
import talib as ta
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- データ取得 ---
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]  # マルチインデックス対策

# --- RSI計算 ---
n1, n2 = 14, 28
rsiS = ta.RSI(price, timeperiod=n1)
rsiL = ta.RSI(price, timeperiod=n2)

# --- シグナル ---
entries = rsiS > rsiL
exits = rsiS < rsiL
entries = entries.fillna(False)
exits = exits.fillna(False)

# --- バックテスト実行 ---
pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# --- チャート構築（2段構え） ---
main_fig = pf.plot()  # vectorbt標準チャート
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.7, 0.3], vertical_spacing=0.05,
                    subplot_titles=("Price + Trades", "RSI"))

# メインチャート追加（上段）
for trace in main_fig.data:
    fig.add_trace(trace, row=1, col=1)

# RSI追加（下段）
fig.add_trace(go.Scatter(x=price.index, y=rsiS, mode='lines', name=f'RSI {n1}', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=price.index, y=rsiL, mode='lines', name=f'RSI {n2}', line=dict(color='red')), row=2, col=1)

# 軸設定
fig.update_yaxes(title_text="RSI", range=[0, 100], row=2, col=1)
fig.update_layout(height=800, showlegend=True, title_text="RSI Cross Strategy (Jupyter表示対応)")

# ✅ ここでJupyterに直接表示（iframeレンダラー使用）
fig.show()


/var/folders/x3/wccbb5_s6s183_4fl9f6ybdc0000gn/T/ipykernel_57888/2076355835.py:9: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

